In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,name,Criterion 1,Criterion 2,Criterion 3,Criterion 4,category
0,Alternative 1,0.374540,0.796543,0.950714,0.183435,Best category
1,Alternative 2,0.731994,0.779691,0.598659,0.596850,Best category
2,Alternative 3,0.156019,0.445833,0.155995,0.099975,Worst category
3,Alternative 4,0.058084,0.459249,0.866176,0.333709,Best category
4,Alternative 5,0.601115,0.142867,0.708073,0.650889,Best category


In [ ]:
thresholds = [
    [0.255905151],
    [0.676961303],
    [0.0551739074],
    [0.324553937]
]

for i in range(1, 5):
  criterion_name = f"Criterion {i}"
  df[criterion_name] = (df[criterion_name] > thresholds[i-1][0]).astype(int)

df.head()


,name,Criterion 1,Criterion 2,Criterion 3,Criterion 4,category
0,Alternative 1,1,1,1,0,Best category
1,Alternative 2,1,1,1,1,Best category
2,Alternative 3,0,0,1,0,Worst category
3,Alternative 4,0,0,1,1,Best category
4,Alternative 5,1,0,1,1,Best category


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df['category'] = df['category'].map({'Best category': 1, 'Worst category': 0})

X = df[['Criterion 1', 'Criterion 2', 'Criterion 3', 'Criterion 4']].values
y = df['category'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

from sklearn.svm import LinearSVC

svm_classifier = LinearSVC(fit_intercept=False)
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X)

print("Accuracy:", accuracy_score(y, y_pred))
print(classification_report(y, y_pred))


Accuracy: 0.97
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        24
           1       0.96      1.00      0.98        76

    accuracy                           0.97       100
   macro avg       0.98      0.94      0.96       100
weighted avg       0.97      0.97      0.97       100



In [ ]:
import pandas as pd
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X, y, 'dataset_svmlight_format.txt')

print("Successfully created dataset_svmlight_format.txt")

In [ ]:
import numpy as np

w = svm_classifier.coef_[0]

In [ ]:
w

array([-0.80995702,  1.47738715, -0.01116209,  1.64803547])

In [ ]:

df = df.loc[~(df[['Criterion 1', 'Criterion 2', 'Criterion 3', 'Criterion 4']] == 0).all(axis=1)]


In [ ]:
# prompt: I want to check for each datapoint if w*X >= 1 or not and add it to df

df['w*X'] = (df[['Criterion 1', 'Criterion 2', 'Criterion 3', 'Criterion 4']].values @ w)
mn = min(df['w*X'].loc[~(df['w*X'] < 0)])
w = w/mn
print(w)
df['w*X'] /= mn
df['w*X >= 1'] = (df['w*X'] >= 1).astype(int)
df['check'] = (df['w*X >= 1'] == df['category']).astype(int)
print(df['check'].value_counts())
df.head()


[-1.23418629  2.25119472 -0.01700843  2.5112231 ]
check
1    97
0     3
Name: count, dtype: int64


,name,Criterion 1,Criterion 2,Criterion 3,Criterion 4,category,w*X,w*X >= 1,check
0,Alternative 1,1,1,1,0,1,1.000000,1,1
1,Alternative 2,1,1,1,1,1,3.511223,1,1
2,Alternative 3,0,0,1,0,0,-0.017008,0,1
3,Alternative 4,0,0,1,1,1,2.494215,1,1
4,Alternative 5,1,0,1,1,1,1.260028,1,1
